In [1]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "phrases_to_highlight.json"

In [2]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output
import json

In [3]:
# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    toHighlight = json.load(f)

In [4]:
isOnMainTable = welcomeUser(name)

Welcome, Jenna Schabdach
You currently have 123 ungraded reports to work on.


In [5]:
batchSize = 25

for i in range(batchSize):
    clear_output()
    print(i)
    if isOnMainTable:
        markOneReportSQL(name, toHighlight)
    else:
        markSelfEvalReportSQL(name, toHighlight)


24
a
BRAIN MRI WITHOUT CONTRAST CLINICAL INDICATION: Seizure TECHNIQUE: Sagittal 3D T1 gradient echo with axial reformations axial and angled coronal TSE T2 axial and angled coronal FLAIR arterial spin labeled perfusion imaging axial susceptibility weighted imaging and axial diffusion tensor imaging were performed on a 3.0 Tesla system. COMPARISON: CT head January 19 2018 FINDINGS: THE VENTRICLES AND SULCI ARE NORMAL IN SIZE AND CONFIGURATION. There is faint T2/FLAIR HYPERINTENSity in the white matter near the right occipital horn (for example series 11 image 11-12) unclear if this is clinically significant. This finding can be non-seizure population as well. No parenchymal MASS lesion or abnormal fluid collection is identified. There is nO MIDLINE SHIFT MASS EFFECT OR ACUTE INFARCTION. THE HIPPOCAMPI ARE SYMMETRIC AND NORMAL IN SIZE SIGNAL AND INTERNAL ARCHITECTURE. THE POSTERIOR FOSSA STRUCTURES ARE NORMAL. THE CEREBELLAR TONSILS TERMINATE ABOVE THE LEVEL OF THE FORAMEN MAGNUM. THE V

Assign a SLIP rating to this report (0 do not use/1 maybe use/2 definitely use):  1


Please confirm your grade by reentering it OR enter a revised value to change the grade:  1


Saving your grade of 1 for this report.
Grade saved. Run the cell again to grade another report.


In [ ]:
# GENERIC: If/when you run out of reports, you can uncomment the following line 
#   and run this cell to get 100 more reports from the database
# getMoreReportsToGrade(name)

# SLIP Psych Prospective: If/when you run out of reports, you can uncomment the 
#  following line and run this cell to get N more reports from the list of reports
#  filtered to belong to your desired cohort
df = pd.read_csv('~/arcus/shared/data/slip_2023_2-5_psych_prospective.csv')
cohortProcIds = list(df['proc_ord_id'].values)
N = 50
addReportsFromListForUser(cohortProcIds, name, maxToAdd=N)